# RAG class

In [1]:
import openai
import pandas as pd
from langchain_openai import AzureChatOpenAI
import os
from configparser import ConfigParser
import shutil
c = ConfigParser()
c.read(r"C:\workspace\APIKEY_personal.ini")

['C:\\workspace\\APIKEY_personal.ini']

In [2]:
rag_config = {
    "api_key":c["AZURE_4o-mini"]["API_KEY"],
    "api_base":c["AZURE_4o-mini"]["OPENAI_API_BASE"],
    "api_version":c["AZURE_4o-mini"]["OPENAI_API_VERSION"],
    "deployment": c["AZURE_4o-mini"]["CHATGPT_MODEL"],
    "langsmith_key": c["KEY"]["LANGSMITH_PERSONAL"]
}

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import AzureChatOpenAI,OpenAI
from langchain.document_loaders import WebBaseLoader,PyPDFLoader
from langchain_community.document_loaders.word_document import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser
import shutil
import os

class RAG:
    def __init__(self,project_name="default_name",config=None,chromadb_folder="./chroma_db",chunkSize=2000,overlap=200) -> None:
        # save config settings to connect to Azure 4o-mini
        self.config = config
        # save project_name
        self.project_name = project_name
        # chromadb path
        self.chromadb_folder = chromadb_folder
        ###### some constants
        # chunksize
        self.chunksize = chunkSize
        # overlap
        self.overlap = overlap
        # embedding model
        # self.model_id = "BAAI/bge-base-en-v1.5" - although this is meant to be really good, doesn't really perform very well!
        self.model_id = "sentence-transformers/all-MiniLM-L6-v2"
        # Set USER_AGENT environment variable
        os.environ["USER_AGENT"] = "myRAGClass/1.0 - " + self.project_name
        # set the openai api keys
        os.environ["OPENAI_API_KEY"] = self.config["api_key"]
        os.environ["OPENAI_API_VERSION"] = self.config["api_version"]
        os.environ["AZURE_OPENAI_ENDPOINT"] = self.config["api_base"]

        # create embedding function
        self.createEmbeddingFunction()

        # create reference to ChromaDB
        self.vectorstore = Chroma(persist_directory=self.chromadb_folder, embedding_function=self.embedding_function)

        # function to split the text into chuncks
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=self.chunksize, chunk_overlap=self.overlap)

    @staticmethod
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    def add_split_text(self,split_doc):
        # Create a unique index on the chunks
        last_page_id = None
        current_chunk_index = 0

        for chunk in split_doc:
            source = chunk.metadata.get("source")
            page = chunk.metadata.get("page")
            if page == last_page_id:
                current_chunk_index += 1
            else:
                current_chunk_index = 0
            last_page_id = page

            chunk_id = f"{source}:{page}:{current_chunk_index}"
            # add chunk id
            chunk.metadata["id"] = chunk_id
        
        # add chunk if it does not exist
        existing_items = self.vectorstore.get(include=[])
        existing_ids = set(existing_items["ids"])
        print(f"Number of existing documents in DB: {len(existing_ids)}")

        # only add documents that don't exist
        new_chunks = []
        for chunk in split_doc:
            if chunk.metadata["id"] not in existing_ids:
                new_chunks.append(chunk)
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]

        if len(new_chunks) > 0:
            self.vectorstore.add_documents(new_chunks,ids=new_chunk_ids)
        else:
            print("Document was in the database already")


    def add_documents(self,docs):
        # loop through references, check whether it is a website, pdf or docx
        for doc in docs:
            if doc.lower().endswith(".pdf"):
                self.add_pdf(doc)
            elif doc.lower().endswith(".docx"):
                self.add_docx(doc)
            elif doc.startswith("http"):
                self.add_website(doc)
            else:
                # print out message that we couldn't add the document
                print(f"Could not add {doc} to the database")
        
    def add_website(self,webpath):
        # load website
        loader = WebBaseLoader(web_paths=(webpath,))
        loader.requests_kwargs = {"verify": False}
        docs = loader.load()
        # split the text into chuncks
        splits = self.text_splitter.split_documents(docs)
        # add the split text to the database
        self.add_split_text(splits)
        print(f"Added {webpath} to the database")

    def add_pdf(self,pdfpath):
        # load pdf
        loader = PyPDFLoader(pdfpath)
        doc = loader.load()
        # split the text into chuncks
        splits = self.text_splitter.split_documents(doc)
        # add the split text to the database
        self.add_split_text(splits)
        print(f"Added {pdfpath} to the database")

    def add_docx(self,filepath):
        doc = Docx2txtLoader(filepath).load()
        # split the text into chuncks
        splits = self.text_splitter.split_documents(doc)
        # add the split text to the database
        self.add_split_text(splits)
        print(f"Added {filepath} to the database")


    def createEmbeddingFunction(self):
        self.embedding_function = HuggingFaceEmbeddings(model_name=self.model_id, 
                                                        model_kwargs={'device': 'cpu',
                                                                      "trust_remote_code":True},
                                                        encode_kwargs = {"max_seq_length": 512})


    def clearDatabase(self):
        # remove the existing RAG database
        shutil.rmtree(self.chromadb_folder)

    def setupLLM(self,local=False,k=10):
        self.prompt = hub.pull("rlm/rag-prompt",api_key=self.config["langsmith_key"])
        self.k = k
        # get retriever
        self.retriever = self.vectorstore.as_retriever(k=self.k)
        #self.retriever = self.vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 20})

        if local == True:
            self.llm = OpenAI(base_url="http://localhost:1234/v1", 
                              api_key="not-needed",
                              max_tokens=6000,
                              temperature=0,
                              max_retries=2)
        else:
            self.llm = AzureChatOpenAI(
                deployment_name=self.config["deployment"],
                model_name=self.config["deployment"], 
                temperature=0,
                max_tokens=None,
                timeout=None,
                max_retries=2)

        self.rag_chain = (
            {"context": self.retriever | self.format_docs, "question": RunnablePassthrough()}
            | self.prompt
            | self.llm
            | StrOutputParser())
    
    def invoke(self,question):
        print("invoking...")
        result = self.rag_chain.invoke(question)
        print(result)
        print("Complete")
        return result


c:\Users\TristramArmour\anaconda3\envs\learning\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
USER_AGENT environment variable not set, consider setting it to identify your requests.


Test website search

In [4]:
webpath = r"https://gamefaqs.gamespot.com/ds/937281-dragon-quest-ix-sentinels-of-the-starry-skies/faqs/60437"
r = RAG(project_name="DQ9_walkthrough_test",config=rag_config,chromadb_folder="./chroma_db_dq9")
r.add_documents([webpath])


C:\Users\TristramArmour\AppData\Local\Temp\ipykernel_32588\1868588418.py:129: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  self.embedding_function = HuggingFaceEmbeddings(model_name=self.model_id,
c:\Users\TristramArmour\anaconda3\envs\learning\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\TristramArmour\anaconda3\envs\learning\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in ver

Number of existing documents in DB: 0
Added https://gamefaqs.gamespot.com/ds/937281-dragon-quest-ix-sentinels-of-the-starry-skies/faqs/60437 to the database


In [5]:
r.setupLLM(local=True)

In [6]:
a = r.invoke("How long will it take for someone to finish DQ9?")

invoking...
 I don't know how long it will take for someone to finish DQ9. The main story is about 35-40 hours, but the post-game content is very large and might take forever to finish everything.
Complete


In [7]:
a = r.invoke("How do I get the best weapons in DQ9?")

invoking...
 To get the best weapons in DQ9, you should focus on purchasing items from the "ITEM NAME" column with the highest price. The top-priced items are likely to have higher stats and better effects. In this case, the top-priced items are Razor-wing Boomerang (9100 G), Potshot Bow (9200 G), and Blowy Bow (12000 G).
Complete


In [8]:
del r

#### Test RAG setup with DOCX

In [9]:
data_path = r"C:\Users\TristramArmour\OneDrive - Innovisk\Documents\BusinessDev\AqPC_LLM_policydoc\PRB ACOM9845_1 03.23.docx"
r = RAG(project_name="PolicyDoc",config=rag_config,chromadb_folder="./chroma_db_policydoc",chunkSize=2000,overlap=200)


c:\Users\TristramArmour\anaconda3\envs\learning\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
r.add_docx(data_path)

Number of existing documents in DB: 0
Added C:\Users\TristramArmour\OneDrive - Innovisk\Documents\BusinessDev\AqPC_LLM_policydoc\PRB ACOM9845_1 03.23.docx to the database


In [11]:
r.setupLLM(local=True)

In [12]:
cd = """A forensic investigation undertaken by Hawkins has concluded at this fried chicken restaurant the cause of the fire was the self-combustion of oily tea towels that had just been tumble dried."""

r.invoke(f"Is this claim covered under the policy? '{cd}'")

invoking...
 No, this claim is not covered under the policy because it involves self-combustion of oily tea towels that had just been tumble dried, which is excluded under the condition precedent for Tumble Dryer usage.
Complete


' No, this claim is not covered under the policy because it involves self-combustion of oily tea towels that had just been tumble dried, which is excluded under the condition precedent for Tumble Dryer usage.'

We did it - we needed k to be qute large, say 10 and increase the chunksize to 2000 from the default 1000. We also used this embedding model - https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

Also increased the max sequence length to 512 tokens, this relates to roughly 2000 characters which seems to be better for RAG applications.